In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup,NavigableString, Tag
import time
import requests
import re
import csv
from selenium.webdriver.common.by import By
import pandas as pd
import os

headers = {'content-type': 'text/html; charset=UTF-8','user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36(KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36'
}

driver = webdriver.Chrome()
driver.get("https://www.money101.com.tw/%E4%BF%A1%E7%94%A8%E5%8D%A1/%E7%B2%BE%E9%81%B8%E6%8E%A8%E8%96%A6")

time.sleep(5) #暫停

#soup = BeautifulSoup(driver.page_source, 'html.parser')
#print(soup.prettify())  #輸出排版後的HTML內容

button= driver.find_element(By.XPATH, "/html/body/div[1]/div/nav/div[1]/div[2]/ul/div[3]/li/label") #
if button.is_displayed() and button.is_enabled():
    button.click()
#button.click()
time.sleep(2)
        
soup = BeautifulSoup(driver.page_source, 'html.parser') #把script變成美麗湯可讀

result = soup.select('html body div:nth-of-type(1) div nav div:nth-of-type(1) div:nth-of-type(2) ul div:nth-of-type(3) li div div')
#print(result)
ranking_link = []
ranking_title = []
for item in result:
    link = item.find('a')
    if link:
        ranking_link.extend(['https://www.money101.com.tw' + a.get('href') for a in item.find_all('a')])
    span = item.find('span')
    if span:
        rank = span.text
        ranking_title.append(rank)
        
ranking_title.pop(0)
ranking_title.pop(0)
ranking_title.pop(0)
ranking_title.pop(0)
ranking_title.pop(0)

ranking_link.pop(0)
ranking_link.pop(0)
ranking_link.pop(0)

#print(ranking_link)
#print(ranking_title)
#print(len(ranking_link))
#print(len(ranking_title))


for k in range(len(ranking_link)):

    if ranking_title[k] == '繳稅優惠':
        continue
#    driver = webdriver.Chrome()
    driver.get(ranking_link[k])
    time.sleep(5) #暫停
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    #print(soup.prettify())  #輸出排版後的HTML內容
    
    
    card_name = []
    card_link = []
    cards = soup.find_all('div',class_="flex items-center justify-between")
    for card in cards:
        title = card.find('h2',class_='pr-1')
        link = card.find('a').get('href')
        card_name.append(title.text)
        print('☆'+title.text)
        print('https://www.money101.com.tw'+link+'\n')
        card_link.append('https://www.money101.com.tw'+link)
        
    #單卡
    #最低年齡要求
    age_request = []
    #最低年薪要求
    salary_request = []
    #國外刷卡手續費
    fee = []
    #年費
    annual_fee = []
    #循環信用利率
    rate = []
    #信用卡發卡機構
    agency = []
    #card_name = []
    card_list2 = ['最低年齡要求','最低年薪要求','國外刷卡手續費','年費','循環信用利率','信用卡發卡機構']
        
    for j in range(len(card_link)):
    
        driver.get(card_link[j]) 
        
        time.sleep(5) #暫停
        
        soup = BeautifulSoup(driver.page_source, 'html.parser') #前往第一張卡的網頁
        
        card_name[j] = card_name[j].replace(" ", "").replace("\n", "")
        print(card_name[j])
        
        result1 = soup.find_all('div', class_="box-border inline-block w-2/5 pr-2 align-top")#標題        
        card_list = []    #每張卡的標題（不一定是六個）    
        for result6 in result1:    
            card_list.append(result6.text)
            print(result6.text)
        
        result2 = soup.find_all('div', class_="inline-block w-1/2") #內文
        n=0
        for result in result2:
            if card_list[n] == "最低年齡要求":
                age_request.append(result.text) 
            elif card_list[n] == "最低年薪要求":
                salary_request.append(result.text)
            elif card_list[n] == "年費":
                fee.append(result.text)
            elif card_list[n] == "國外刷卡手續費":
                annual_fee.append(result.text)
            elif card_list[n] == "循環信用利率":
                rate.append(result.text)
            elif card_list[n] == "信用卡發卡機構":
                agency.append(result.text)       
            print(card_list[n]+":"+result.text+'\n')
            n+=1
            
        if len(age_request) != j+1:
            age_request.append('不適用')
        if len(salary_request) != j+1:
            salary_request.append('不適用')
        if len(fee) != j+1:
            fee.append('無')
        if len(annual_fee) != j+1:
            annual_fee.append('不適用')
        if len(rate) != j+1:
            rate.append('不適用')
        if len(agency) != j+1:
            agency.append('不適用')
            
        
        
        
        #table
        #可能沒有
        table = soup.find('div', class_='min-w-0 flex-1 md:ml-2')
        if table is not None:
            tables = table.find_all('table', style = 'width:100%') #如果有table
            title_list=[] #優惠csv
            text_list =[] #優惠csv
            if tables and ranking_title[k]== '台新銀行 台新街口聯名卡':
                title_list.append('活動一')
                title_list.append('活動二')
            #print(table)
            if tables:
                for table in tables:
                    if table != None  and ranking_title[k]== '滙豐銀行 匯鑽卡':
                        results = table.find_all('tr')
                        i=0
                        header =[]
                        a = []
                        #b = []
                        for result in results:
                            r = result.find_all('td')
                            if i == 0:
                                for r in result:
                                    #print(r.text)
                                    header.append(r.text)
                                #print(i)
                            elif i ==1:
                                for r in result:
                                    #print(r.text)
                                    a.append(r.text)
                                #print(i)
                        #    elif i ==2:
                        #        for r in result:
                                    #print(r.text)
                        #            b.append(r.text)
                                #print(i)
                            i=i+1
                        
                        for i in header:
                            if ' ' in header:
                                header.remove(' ')
                                
                        for i in a:
                            if ' ' in a:
                                a.remove(' ')
                                
                        #for i in b:
                        #    if ' ' in b:
                        #        b.remove(' ')
                        #b[0] = b[0][0:4]
                        print(header)
                        print(a)
                        #print(b)
                        #d = [a]
                        #df = pd.DataFrame(d, columns = header)
                        #print(df)
                        title_list.append(header[1]+'的'+a[0])
                        title_list.append(header[2]+'的'+a[0])
                        text_list.append(a[1])
                        text_list.append(a[2])
                        print(header[1]+'的'+a[0]+': '+a[1])
                        print(header[2]+'的'+a[0]+': '+a[2])
                    #   print(header[1]+'的'+b[0]+': '+b[1])
                    #   print(header[2]+'的'+b[0]+': '+b[2])
                    elif table != None  and ranking_title[k]== '台新銀行 台新街口聯名卡':
                        table_header = []
                        table_text = []
                        trs = table.find_all('tr')
                        for tr in trs:
                            ths = tr.find_all('th')
                            tds = tr.find_all('td')
                            if ths !=None:
                                i = 0
                                for th in ths:
                                    if i == 0:
                                        table_header.append(th.text)
                                    elif i == 1:
                                        table_text.append(th.text)
                                    print(th.text)
                                    i+=1
                            else:
                                i = 0
                                for th in ths:
                                    if i == 0:
                                        table_header.append(td.text)
                                    elif i == 1:
                                        table_text.append(td.text)
                                    print(td.text)
                                    i+=1
                        texts=[]
                        for i in  range(len(table_header)):
                            texts.append(table_header[i]+':'+table_text[i])
                        t = '\n'.join(texts)
                        text_list.append(t)
                    
                

        #聯邦銀行 吉鶴卡
        results = soup.find_all('div', class_='min-w-0 flex-1 md:ml-2') #產品特色、注意事項
        for result in results:
            print(result.find('p').text) #”產品特色“ “注意事項”
            result1 = result.find('ul', class_='-my-1') #產品特色大框、注意事項大框
            result2 = result1.find_all('li', class_='my-1 flex gap-1 font-small') #產品特色 勾 注意事項 橫槓
            for re in result2:
                text1 = re.find('div', class_ = 'rich-text w-11/12 md:w-full')
                table = text1.find('table', style = 'width:100%') #如果有table
                if table != None:
                    result3 = text1.find_all('li')
                    for res in result3:
                        print(res.text+'\n')
                        text_list.append(res.text)
                        title_list.append(result.find('p').text)
                else:
                    print(text1.text+'\n')
                    text_list.append(text1.text)
                    title_list.append(result.find('p').text)
        #print(text_list)
        
        results = soup.find_all('div', class_="grow md:grid md:gap-x-6 Section_wide__mQogI")
        for result in results:
            result3 = result.find_all('span', class_="font-small font-bold")
            #print(result2)
            #title_list=[] #小標（消費類型）
            for result in result3:
                title_list.append(result.text)
                #print(result.text+'\n')
            
            for i in range(len(title_list)):
                print(title_list[i])
        
        results = soup.find_all('div', class_="grow md:grid md:gap-x-6 Section_wide__mQogI")
        for result in results:
            result2 = result.find_all('div', class_="flex-1 pl-2 text-right")
            if result2:
                #print(result2)
                text=[] #內文(現金回饋)
                
                for result in result2:
                    text.append(result.text)
                    #print(result.text+'\n')
                    
                text.pop(0)
                for i in range(len(text)):
                    print(text[i])
                    text_list.append(text[i])
            
        card_dict1={
            "標題":title_list,
            "詳情":text_list
        }
        
        df = pd.DataFrame(card_dict1)
        folder_name = 'money101'+ranking_title[k]
        path = "/Users/takoyaki/Downloads/專題爬蟲/money101csv"  #要改路徑
    
        folder_path = os.path.join(path, folder_name)
        if not os.path.isdir(folder_path):
            os.makedirs(folder_path, mode=0o777)
        path1 = str(path+'/'+folder_name+"/") #要改斜線
        #path='/Users/takoyaki/Downloads/專題爬蟲/money101csv/精選推薦卡csv/'
        df.to_csv(path1+"money101"+card_name[j]+"優惠.csv", index=False) #, index=False
        
    card_dict = {
        '卡片名稱': card_name[k],
        card_list2[0]:age_request,
        card_list2[1]:salary_request,
        card_list2[2]:fee,
        card_list2[3]:annual_fee,
        card_list2[4]:rate,
        card_list2[5]:agency
    }    
    
    
    df = pd.DataFrame(card_dict)
    folder_name = 'money101'+ranking_title[k]
    path = "/Users/takoyaki/Downloads/專題爬蟲/money101csv"  #要改路徑

    folder_path = os.path.join(path, folder_name)
    if not os.path.isdir(folder_path):
        os.makedirs(folder_path, mode=0o777)
    path0 = str(path+'/'+folder_name+"/") #要改斜線
    #path1 = '/Users/takoyaki/Downloads/專題爬蟲/money101csv/精選推薦卡csv/' 
    df.to_csv(path0+"money101"+ranking_title[k]+"基本資訊"+'.csv', index=False)
    
driver.close() 

☆滙豐銀行 匯鑽卡
https://www.money101.com.tw/信用卡/產品/滙豐銀行匯鑽卡?entry=header&from=%E7%B2%BE%E9%81%B8%E6%8E%A8%E8%96%A6

☆聯邦銀行 吉鶴卡
https://www.money101.com.tw/信用卡/產品/聯邦銀行吉鶴卡?entry=header&from=%E7%B2%BE%E9%81%B8%E6%8E%A8%E8%96%A6

☆第一銀行 iLEO卡
https://www.money101.com.tw/信用卡/產品/第一銀行ileo卡?entry=header&from=%E7%B2%BE%E9%81%B8%E6%8E%A8%E8%96%A6

☆中國信託 LINE Pay卡
https://www.money101.com.tw/信用卡/產品/中國信託linepay卡?entry=header&from=%E7%B2%BE%E9%81%B8%E6%8E%A8%E8%96%A6

☆台新銀行 玫瑰Giving卡
https://www.money101.com.tw/信用卡/產品/台新銀行玫瑰giving卡?entry=header&from=%E7%B2%BE%E9%81%B8%E6%8E%A8%E8%96%A6

☆玉山銀行 熊本熊卡
https://www.money101.com.tw/信用卡/產品/玉山銀行熊本熊卡?entry=header&from=%E7%B2%BE%E9%81%B8%E6%8E%A8%E8%96%A6

☆元大銀行 Hello Kitty鑽金聯名卡
https://www.money101.com.tw/信用卡/產品/元大銀行hellokitty鑽金聯名卡?entry=header&from=%E7%B2%BE%E9%81%B8%E6%8E%A8%E8%96%A6

☆台北富邦 J卡
https://www.money101.com.tw/信用卡/產品/台北富邦j卡?entry=header&from=%E7%B2%BE%E9%81%B8%E6%8E%A8%E8%96%A6

☆永豐銀行 幣倍卡
https://www.money101.com.tw/信用卡/產品/永豐銀行幣倍卡?entry=header&from=%E7%B

元大銀行HelloKitty鑽金聯名卡
最低年齡要求
最低年薪要求
國外刷卡手續費
年費
循環信用利率
信用卡發卡機構
最低年齡要求:18 歲

最低年薪要求:依銀行審核條件為主

國外刷卡手續費:1.5 %

年費:正卡NT$1,800、附卡NT$900。申請電子帳單且取消實體帳單之持卡人，於電子帳單申請期間可享免年費之優惠。

循環信用利率:信用卡循環信用年利率：2.92~15%(基準日：106/4/1)預借現金手續費：每筆預借現金金額x3.5%+NT$100其他費用請至元大銀行官網查詢

信用卡發卡機構:VISA

產品特色
超高回饋（無消費門檻，回饋無上限） 國外消費回饋：2.2% 國內消費回饋：1.2%

三款超萌Hello Kitty卡面任選 時尚白/甜心粉/個性藍

暢遊全球免費WiFi上網該次海外行程之海外消費累計滿NT$30,000，享該次海外行程最多5天免費租用行動上網分享器

Mastercard鈦金商務卡優惠機場接送、貴賓室優惠詳參網頁說明https://ytbk.tw/FrMniy

注意事項
 定義國內/外一般消費使用行動支付付款同樣享有現金回饋 一般支付不包含：自動加值交易、信用卡年費、循環利息、信用卡相關手續費、預借現金、各式貸款、餘額代償、申購基金所生之帳款及其各項費用、利息、各項代繳帳款及其費用、公務機關信用卡繳費平台、電子化繳費稅處理平台、醫指付APP、退貨交易、申請退稅至信用卡之交易、爭議款、其他非消費性款項、或透過「聯合信用卡處理中心」提供之信用卡小額支付特約商店之消費、icash Pay等電支工具綁定元大信用卡之消費、境外投資交易平臺之交易及其他元大銀行公告排除之項目或依其他特別約定條款不予計入者。 

產品特色
產品特色
產品特色
產品特色
注意事項
國內消費
國外消費
行動支付回饋
1.2%
2.2%
1.2%
台北富邦J卡
最低年齡要求
最低年薪要求
年費
國外刷卡手續費
循環信用利率
信用卡發卡機構
最低年齡要求:18 歲

最低年薪要求:NT$200,000

年費:正卡年費NT$1,800；首年免年費，次年消費正附卡合併達6萬或12次免年費。

國外刷卡手續費:1.5 %

循環信用利率:信用卡循環年利率：台北富邦銀行指數型房貸基準利率(I)+加碼利率(該加碼利率區間為0.62%~14.00%，基準日為108.

第一銀行iLEO卡
最低年齡要求
最低年薪要求
年費
國外刷卡手續費
循環信用利率
信用卡發卡機構
最低年齡要求:18 歲

最低年薪要求:150000

年費:正卡1,200元，首年免年費，只要年消費次數1次（不限金額）並綁定電子帳單，即免次年年費。

國外刷卡手續費:1.5%

循環信用利率:差別循環信用年利率為一銀放款基準利率+2.31%～9.31%｜差別循環信用年利率依電腦評定，最高為15%．利率之基準日為2015年9月1日｜預借現金手續費：預借金額X3.5%（最低新臺幣100元/3.5美元/400日圓/3歐元計）．其他收費項目及權益請上網查詢

信用卡發卡機構:Mastercard

產品特色
指定行動支付享4%現金回饋，新戶再加碼6%現金回饋  活動期間：2023/7/1-2023/12/31 活動任務：活動期間以本行帳戶扣繳iLEO卡費成功，享以下優惠      表一           適用活動   指定行動支付回饋率     活動說明   現金  回饋上限   台灣Pay   LINE Pay 街口支付     合計最高回饋   /   10.5%   10%     新戶加碼最高(見下表二)   /   6%   6%         綁定行動支付最高     (1+2+3+4)   /   4.5%   4%         1.一般消費回饋   無上限   0.5%   0.5%         2.指定通路加碼1.5%   每期帳單 500元   /   1.5%         3.行動支付綁定加碼     (單筆滿500元且自扣卡費成功)   每期帳單 200元   2%   2%         4.台灣Pay加碼   每月 100元   2%   /     表二           新戶適用活動   指定行動支付回饋率     活動說明   現金  回饋上限   台灣Pay   LINE Pay 街口支付     合計新戶加碼最高   /   6%   6%         綁定iLEO數位帳戶     扣繳卡費享1％   首六期帳單 每期帳單 100元   適用   適用         線上辦卡享5% (單筆消費1,000元以上)   每戶 250元   適用   適用     

海外實體店家消費享3%現金回饋，新戶再加碼8

滙豐銀行現金回饋御璽卡
最低年齡要求
最低年薪要求
國外刷卡手續費
年費
循環信用利率
信用卡發卡機構
最低年齡要求:18 歲

最低年薪要求:NT$ 250,000

國外刷卡手續費:1.5 %

年費:御璽卡新台幣2000元。免年費條件：新戶享首年免年費；使用非紙本帳單(電子帳單或行動帳單)、前一年消費滿8 萬或 12 次

循環信用利率:信用消費及預借現金適用之循環信用利率：5.68%~15.00%；每筆預借現金手續費：新台幣100元+預借現金金額x3.5%；其他費用查詢請洽匯豐銀行網站；循環利率基準日為104年9月1日。詳情見滙豐官網。

信用卡發卡機構:VISA

產品特色
大方現金回饋：國內消費享現金回饋1.22%、海外享現金回饋2.22%．無消費門檻、無級距門檻、回饋無上限回饋終身有效，自動折抵帳單金額海外不含歐盟實體店面消費

厲害再升級，2種回饋方式任您隨心換  折抵帳單金額： 每期自動折抵，或帳戶累積後一次折抵 兌換航空哩程：回饋金1元 = 2 哩；無兌換門檻， 適用於華航（天合聯盟）／亞洲萬里通／新航（星空聯盟）/長榮航空（星空聯盟）哩程，透過3大航空聯盟可兌換航空公司獎勵機票 

美食最優買1送1：指定飯店美食包括長榮桂冠(台中) 、台糖長榮(台南) 、及知名美食Cold Stone皆享最優買1送1(等同50%折扣)

全方位旅遊保障：享2,000萬公共運輸交通工具旅遊平安險10萬海外全程意外傷害醫療保險班機延誤及行李延誤遺失等旅遊不便險

注意事項
不分新舊戶均享首年免年費，申辦行動/電子帳單即享終身免年費

定義國內/外一般消費使用行動支付付款同樣享有現金回饋

產品特色
產品特色
產品特色
產品特色
注意事項
注意事項
國內消費
國外消費
加油消費
繳保費
行動支付回饋
1.22%
5%
1.22%
1.22%
2.22%
玉山銀行熊本熊卡
最低年齡要求
最低年薪要求
國外刷卡手續費
年費
循環信用利率
信用卡發卡機構
最低年齡要求:18 歲

最低年薪要求:不適用

國外刷卡手續費:熊本熊一般信用卡(向右右版及最愛吃版)：1.5%；熊本熊日圓雙幣卡(向左左版及很友好版)：免收1.5%國外交易服務費。

年費:玉山熊本熊正卡3,000元/卡(首年免年費)，附卡免年費；使用帳單e化期間享免年費優惠；每年有消費，年年免年費。

循環信用利率

元大銀行HelloKitty鑽金聯名卡
最低年齡要求
最低年薪要求
國外刷卡手續費
年費
循環信用利率
信用卡發卡機構
最低年齡要求:18 歲

最低年薪要求:依銀行審核條件為主

國外刷卡手續費:1.5 %

年費:正卡NT$1,800、附卡NT$900。申請電子帳單且取消實體帳單之持卡人，於電子帳單申請期間可享免年費之優惠。

循環信用利率:信用卡循環信用年利率：2.92~15%(基準日：106/4/1)預借現金手續費：每筆預借現金金額x3.5%+NT$100其他費用請至元大銀行官網查詢

信用卡發卡機構:VISA

產品特色
超高回饋（無消費門檻，回饋無上限） 國外消費回饋：2.2% 國內消費回饋：1.2%

三款超萌Hello Kitty卡面任選 時尚白/甜心粉/個性藍

暢遊全球免費WiFi上網該次海外行程之海外消費累計滿NT$30,000，享該次海外行程最多5天免費租用行動上網分享器

Mastercard鈦金商務卡優惠機場接送、貴賓室優惠詳參網頁說明https://ytbk.tw/FrMniy

注意事項
 定義國內/外一般消費使用行動支付付款同樣享有現金回饋 一般支付不包含：自動加值交易、信用卡年費、循環利息、信用卡相關手續費、預借現金、各式貸款、餘額代償、申購基金所生之帳款及其各項費用、利息、各項代繳帳款及其費用、公務機關信用卡繳費平台、電子化繳費稅處理平台、醫指付APP、退貨交易、申請退稅至信用卡之交易、爭議款、其他非消費性款項、或透過「聯合信用卡處理中心」提供之信用卡小額支付特約商店之消費、icash Pay等電支工具綁定元大信用卡之消費、境外投資交易平臺之交易及其他元大銀行公告排除之項目或依其他特別約定條款不予計入者。 

產品特色
產品特色
產品特色
產品特色
注意事項
國內消費
國外消費
行動支付回饋
1.2%
2.2%
1.2%
台北富邦J卡
最低年齡要求
最低年薪要求
年費
國外刷卡手續費
循環信用利率
信用卡發卡機構
最低年齡要求:18 歲

最低年薪要求:NT$200,000

年費:正卡年費NT$1,800；首年免年費，次年消費正附卡合併達6萬或12次免年費。

國外刷卡手續費:1.5 %

循環信用利率:信用卡循環年利率：台北富邦銀行指數型房貸基準利率(I)+加碼利率(該加碼利率區間為0.62%~14.00%，基準日為108.

...

星展銀行飛行鈦金卡
最低年齡要求
最低年薪要求
國外刷卡手續費
年費
循環信用利率
信用卡發卡機構
最低年齡要求:20 歲

最低年薪要求:NT$ 220,000

國外刷卡手續費:1.5 %

年費:鈦金卡NT$1,200元。免年費條件：首年免年費，持卡第二年起，凡當年度 正卡年度累計交易次數達12次 或 正卡年度累計消費金額達NT$30,000 或 全年度使用信用卡電子帳單 可免繳次年年費。

循環信用利率:星展銀行(台灣)信用卡循環利率區間：5.99%~14.99%循環利率基準日：民國107年2月1日預借現金手續費：預借現金金額乘以2.5%+NT$ 150。其他費用歡迎至星展銀行(台灣)網站www.dbs.com.tw查詢。

信用卡發卡機構:Mastercard

產品特色
輕鬆兌換飛行積金  海外刷卡每NT$20累積1點飛行積金 國內刷卡每NT$30累積1點飛行積金 持卡期間終身有效 飛行積金可兌換「亞洲萬里通」、「長榮無限萬哩遊」、「新航獎勵計劃KrisFlyer」、「華夏哩程酬賓計劃」四大航空公司的旅遊獎勵計劃*單筆最低兌換門檻為 3,000 點飛行積金，兌換基準以每 1,000 點飛行積金為倍數 

卡友優惠  全球龍騰出行機場貴賓室優惠價 國際機場接機或送機優惠價最低NT$628起 1年3次每次最多5天國際機場外圍停車禮遇 最高新臺幣2千萬公共運輸工具旅遊平安險 全球禮賓尊榮秘書 刷卡消費即時簡訊服務 

回饋須知飛行積金每1,000點可以折抵指定商店NT$100消費款項；每100點可兌換1小時「台灣聯通停車場」、「中興嘟嘟房停車場」、「詮營停車場」、「永固便利停車場」或「福慧網停車場」的免費停車時數

產品特色
產品特色
產品特色
國內消費
國外消費
繳保費
0%
0%
1%
永豐銀行MeCard
最低年齡要求
最低年薪要求
國外刷卡手續費
年費
循環信用利率
信用卡發卡機構
最低年齡要求:18 歲

最低年薪要求:不適用

國外刷卡手續費:1.5 %

年費:正卡年費：御璽卡NT$3000。免年費條件：年度累積消費達NT$15萬 或 不限金額12次，或申請信用卡電子對帳單且取消實體帳單，即可享有次年免年費優惠。

循環信用利率:循環信用利率年息2.74%-15%（基準日2015/8/3），預借現金手續費：預借現金金額x3.5%+NT$100。


滙豐銀行旅人無限卡
最低年齡要求
最低年薪要求
年費
國外刷卡手續費
循環信用利率
信用卡發卡機構
最低年齡要求:18歲

最低年薪要求:250,000元

年費:8000

國外刷卡手續費:1.5

循環信用利率:信用消費及預借現金適用之循環信用利率：5.68%~15.00%；每筆預借現金手續費：新台幣100元+預借現金金額x3.5%；其他費用查詢請洽匯豐銀行網站；循環利率基準日為104年9月1日。詳情見滙豐官網。

信用卡發卡機構:VISA

產品特色
消費回饋國內消費：18元=1旅遊積分海外消費：10元=1旅遊積分

旅遊積分兌換：  15家以上航空公司1:1兌換 三大飯店集團IHG、Accor、Marriot 1:1兌換 5,000旅遊積分可折抵1,500元刷卡金(折抵帳單)

卡友優惠免費機場接送4次/年免費機場貴賓室8次/年機場接送最低NT$420起免費機場停車60天五星美饌最優買1送1

產品特色
產品特色
產品特色
產品特色
產品特色
產品特色
國內消費
國外消費
不適用
不適用
滙豐銀行旅人御璽卡
最低年齡要求
最低年薪要求
年費
國外刷卡手續費
循環信用利率
信用卡發卡機構
最低年齡要求:18歲

最低年薪要求:250,000元

年費:2500

國外刷卡手續費:1.5

循環信用利率:信用消費及預借現金適用之循環信用利率：5.68%~15.00%；每筆預借現金手續費：新台幣100元+預借現金金額x3.5%；其他費用查詢請洽匯豐銀行網站；循環利率基準日為104年9月1日。詳情見滙豐官網。

信用卡發卡機構:VISA

產品特色
消費回饋國內消費：18元=1旅遊積分海外消費：15元=1旅遊積分

旅遊積分兌換：  15家以上航空公司1:1兌換 三大飯店集團IHG、Accor、Marriot 1:1兌換 5,000旅遊積分可折抵1,500元刷卡金(折抵帳單)

卡友優惠免費機場接送2次/年免費機場貴賓室2次/年免費機場停車45天五星美饌最優買1送1

產品特色
產品特色
產品特色
產品特色
產品特色
產品特色
國內消費
國外消費
不適用
不適用
中國信託LINEPay卡
最低年齡要求
最低年薪要求
國外刷卡手續費
年費
循環信用利率
信用卡發卡機構
最低年齡要求:18 歲

最低年薪要求:NT$ 200,000

國外刷卡手續費:1.5 %

年費

台新銀行昇恆昌聯名卡
最低年齡要求
最低年薪要求
國外刷卡手續費
年費
循環信用利率
信用卡發卡機構
最低年齡要求:20 歲

最低年薪要求:不適用

國外刷卡手續費:1.5 %

年費:白金卡NT$1500,普卡NT$300,御璽卡NT$3000/免年費條件：普卡/白金卡 年度不限金額任刷1筆；御璽卡 年度累積消費滿NT$60,000或年度消費次數達6次，可折抵正卡及正卡名下所有附卡次年年費。

循環信用利率:差別循環信用利率：一般消費及預借現金為6.75%~15%，依電腦評等而定；預借現金手續費：預借現金金額x3%，不足NT$100以NT$100計算；循環利率之基準日為104年9月1日；特店分期廣告已揭露各項相關費用總金額且無其他應繳費用，各期分期利率換算總費用年百分率為0%。

信用卡發卡機構:VISA

產品特色
現金回饋國內刷卡消費享0.5%現金回饋國外刷卡消費享最高1.5%現金回饋

昇恆昌獨家優惠持昇恆昌聯名卡至全臺昇恆昌免稅商店刷卡消費，不僅可以用免稅價格擁有優質商品，部份專櫃品牌、香水化妝品、精品配件、進口食品及國產品等商品，還可享免稅部分再 9 折起優惠

年度卡友禮生日當月起算3個月內至昇恆昌店內，以昇恆昌聯名卡消費，加碼0.5%回饋

保費回饋1%無上限以昇恆昌御璽卡刷卡代扣繳保險費，皆享1%回饋無上限

卡友優惠 國際機場貴賓室 機場外圍停車每次最高10天，全年最高2次市區停車每日免費2小時

注意事項
年度累積消費之排除項目詳情見官網

定義  國內/外一般消費使用行動支付付款同樣享有現金回饋 一般消費不包含：歐盟地區實體商店交易、境外投資平台交易、特約商店分期付款交易及單筆消費分期0利率之交易、電子支付儲值卡自動加值、Samsung Pay悠遊卡加值、全聯福利中心消、便利商店、各項代扣繳費用、躉繳保費、投資型保費、三商美邦保費、彈性繳保費、藥安康藥局消費、臺北醫學大學附設醫院費用、各縣市路邊停車費、透過各繳費平台繳納之各項費用、政府規費、違規罰鍰、透過「聯合信用卡處理中心」提供之信用卡小額支付特約商店之消費、預借現金及其手續費、預借現金分期金及其手續費、代償他行信用卡/現金卡/小額信貸等之金額、個人綜合所得稅、所得稅分期付款期金及其手續費、查核定稅、燃料費、違約金、循環利息、基金交易及其手續費、信用卡年費及一般信用卡相關手續費。  



聯邦銀行信用卡
台新銀行信用卡
中國信託信用卡
滙豐銀行信用卡
凱基銀行信用卡
永豐銀行信用卡
遠東銀行信用卡
玉山銀行信用卡
臺灣企銀信用卡
彰化銀行信用卡
星展銀行信用卡
安泰銀行信用卡
國泰世華信用卡
合作金庫信用卡
元大銀行信用卡
台北富邦信用卡
兆豐銀行信用卡
土地銀行信用卡
第一銀行信用卡
上海銀行信用卡
日盛銀行信用卡
臺灣銀行信用卡
新光銀行信用卡
華南銀行信用卡
高雄銀行信用卡
三信銀行信用卡
台中銀行信用卡
華泰銀行信用卡
台灣樂天信用卡


IndexError: list index out of range

In [75]:
#print(len(age_request))
#print(len(salary_request))
#print(len(fee))
#print(len(annual_fee))
#print(len(rate))
#print(len(agency))

129
129
129
128
127
124
